# Music
<img src="Music/Y4_Large_reduc.jpg" align="right" width="64" height="64" />

Zen's Music Playround

# Introduction

This page contains some experiments with Jupyter and playing with my music collection. To play happily, we have divided this page into multiple sections.

These sections are:
* [Setup](#Setup "Load the Music Library class and variables")
* [Get random albums](#Random-Albums "Get 5 random albums")
* [Get random album and cover](#Random-Album-and-Cover "Get a random album and it's track listing.")

For useful help, you can check out:
* The [Jupyter Widget List](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html)


## Setup

Execute the cell below to instantiate the MusicLib class and make it available as *lib*.

In [88]:
from os import listdir
from os.path import join, isdir, exists, basename
from random import sample
from glob import glob
import ipywidgets as widgets
from IPython.core.display import HTML, display
from textwrap import dedent
import codecs

class Templates(object):
    """
    This class defines varios HTML templates as constants from output in
    the MusicLib class.
    """
    @staticmethod
    def get_album():
        """
        Return the HTML for an album template. The template has placeholders
        for 'artist', 'album' and 'tracks'.
        """
        return dedent('''
            <table>
                <tr>
                  <th><b>{artist}</b></th>
                  <th><b>{album}</b></th>
                </tr>
                <tr>
                  <td>{cover}</td>
                  <td>{tracks}</td>
                </tr>            
            </table>
        ''')
    
    @staticmethod
    def get_image():
        """
        Return the HTML for the album cover with a placeholder for
        'base64'
        """
        return '<img alt="Image" src="data:image/png;base64,{base64}"'\
            ' width="300px" height="300px" />'
    
class MusicLib(object):
    """
    Class for fetching information about our music library, as an experiment
    in jupyter
    """
   
    def __init__(self, path):
       self.path = path 
    
    def artists(self):
        """ Return a list of lartists. """
        return [name for name in listdir(self.path) if isdir(join(self.path, name))]

    def get_random_artists(self, number):
        """ Return a random list of *number* artists. """
        artists = self.artists()
        return sample(artists, number)

    def get_random_albums(self, artist, number):
        """ Return a random list of *number* albums by *artist*. """
        albums = self.albums(artist)
        return sample(albums, number)

    def albums(self, artist):
        """ Return a list of albums for the *artist*. """
        path = join(self.path, artist)
        return [name for name in listdir(path) if isdir(join(path, name))]
    
    def get_album_cover(self, artist, album, size=300):
        """
        Return the album cover for the specified album. Return an IPython widget
        if path is False, otherwise it returns the full fiel path.
        """
        path = join(self.path, artist, album)
        pattern = "cover.*"
        matches = glob(join(path, pattern))
        if matches:
            fname = matches[0]
            with open(fname, "rb") as f:
                b64 = codecs.encode(f.read(), 'base64')
                html = Templates.get_image()
                # return HTML(html.format(**{'base64': str(b64, 'ascii')}))
                return html.format(**{'base64': str(b64, 'ascii')})
                
    def get_album_tracks(self, artist, album):
        """ Return a list of the album tracks"""
        def get_name(fname):
            """"Return the nice, cleaned name of the track"""
            return basename(fname)  # [:-4]
        
        path = join(self.path, artist, album)
        pattern = "*.mp3"
        matches = glob(join(path, pattern))        
        return sorted([get_name(f) for f in matches])

    def get_album_summary(self, artist, album):
        """ Return a pretty album summary """
        cover = self.get_album_cover(artist, album)
        items = [widgets.Label("{0}: {1}".format(artist, album))]
        [items.append(widgets.Label(track, width=200)) for track in self.get_album_tracks(artist,
                                                                album)]
        right = widgets.VBox(items, width=300)
        return widgets.HBox([cover, right])

    def get_album_summary_html(self, artist, album, out):
        """ Return a pretty album summary """
        with out:
            data = {'artist': artist,
                    'album': album}
            cover = self.get_album_cover(artist, album)       
            tracks = [track for track in self.get_album_tracks(artist, album)]
            data['tracks'] = "<br/>".join(tracks)
            data['cover'] = cover
            html = HTML(Templates.get_album().format(**data))
            # display(cover)
            display(html)
            

library = "/home/fruitbat/Music/"
lib = MusicLib(library)           

[Back to top](#Introduction "Back the top of the page")

In [83]:
from IPython.display import clear_output
from ipywidgets import Output

out = None

def get_random_album(btn):
    """ Generate and display a random album. """
    global out
    if out is not None:
        out.clear_output(wait=True)
    else:
        out = Output()
        display(out)
        
    artist =  lib.get_random_artists(1)[0]
    album =  lib.get_random_albums(artist, 1)[0]
    lib.get_album_summary_html(artist, album, out)

btn = widgets.Button(description='Display a random album')
btn.on_click(get_random_album)
btn

## Random Albums

In [14]:
artists = lib.get_random_artists(5)
for artist in artists:
    albums = lib.albums(artist)
    print("{0}: {1} albums".format(artist, len(albums)))
    [print("    {0}".format(album)) for album in albums]

Gary Moore: 1 albums
    Back On The Streets
Infected Mushroom: 9 albums
    Return to the Sauce
    I'm The Supervisor
    Vicious Delicious
    Converting Vegetarians - The Other Side
    Legend Of The Black Shawarma
    Classical Mushroom
    B.P. Empire
    Army of Mushrooms
    Converting Vegetarians - Trance Side
George Michael: 1 albums
    Older
Def Leppard: 2 albums
    Retro Active
    Vault
Beck: 1 albums
    Odelay!


[Back to top](#Introduction "Back the top of the page")

## Random Album and Cover

In [91]:
# Get a random artist, a random album and list the tracks

artist =  lib.get_random_artists(1)[0]
album =  lib.get_random_albums(artist, 1)[0]
print("{0}: {1}".format(artist, album))
[print(track) for track in lib.get_album_tracks(artist, album)]
HTML(lib.get_album_cover(artist, album, 200))


Dr Dre: The Chronic
01 - Lolo (Intro ) Feat Xzibit & tray-Dee.mp3
02 - The Watcher.mp3
03 - You Feat Devin a-k-a The Dude & Snoop.mp3
04 -  Still Dre feat Snoop.mp3
05 - Big Ego's feat Hitman.mp3
06 - Xxplosive feat Hitman, Kurupt, Nate Dogg & Six-Two.mp3
07 - What's the Difference feat Eminem & Xzibit.mp3
08 - Bar One feat Traci Nelson, Ms. Roq & Eddie Griffin.mp3
09 - Light Speed feat Hitman.mp3
10 - Forgot About Dre. feat Enimen.mp3
11 - The Next Episode feat Snoop.mp3
12 - Let's Get High feat Hitman, Kurupt & Ms.Roq.mp3
13 - Bitch Niggaz feat Snoop, Hitman & Six-Two.mp3
14 - The Car Bomb Feat Mel-Man & Charis Henry.mp3
15 - Murder Ink feat Hitman -& Mis.Roq.mp3
16 - Edu-cation feat Eddie Grafin.mp3
17 - Some L.A Niggaz  feat DeFari, Xzibit, Knoc-Tur'nal, Tim.mp3
18 - Pause For Porn feat Jake Speed.mp3
19 - House Wife feat Kurupt & Hitman.mp3
20 - Ackrite feat Htiman.mp3
21 - Bang Bang feat Knoc-Turn'al & htiman.mp3
22 - The Message Feat Mary J.Blige & Rell.mp3


[Back to top](#Introduction "Back the top of the page")

In [25]:
%whos

Variable           Type                          Data/Info
----------------------------------------------------------
HTML               type                          <class 'IPython.core.display.HTML'>
MusicLib           type                          <class '__main__.MusicLib'>
Output             MetaHasTraits                 <class 'ipywidgets.widgets.widget_output.Output'>
Templates          type                          <class '__main__.Templates'>
basename           function                      <function basename at 0x7f0d7ac07620>
btn                Button                        <ipywidgets.widgets.widge<...>object at 0x7f0d606df400>
clear_output       function                      <function clear_output at 0x7f0d77264378>
dedent             function                      <function dedent at 0x7f0d77f29048>
display            function                      <function display at 0x7f0d77259a60>
exists             function                      <function exists at 0x7f0d7ac5dae8>
get_